### **Step 1: Load and Clean Data**

### **Genders Are male & female**

### **Match Types Are: ODI, T20, Test, MDM, ODM**



In [1]:
import os
import pandas as pd

base_path = r"D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary"

# Get genders from folders inside match_summary
genders = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]

# Show gender options
print("Select Gender:")
for idx, gender in enumerate(genders, start=1):
    print(f"{idx}. {gender.capitalize()}")
gender_choice = int(input("Enter choice: ").strip()) - 1
gender = genders[gender_choice]

# Get match types from filenames in the selected gender folder
gender_path = os.path.join(base_path, gender)
match_types = []
for file in os.listdir(gender_path):
    if file.endswith("_match_summary_cleaned.csv"):
        match_type = file.split("_")[0]  # e.g., 'odi' from 'odi_men_match_summary_cleaned.csv'
        if match_type not in match_types:
            match_types.append(match_type)

# Show match type options
print("\nSelect Match Type:")
for idx, mtype in enumerate(match_types, start=1):
    print(f"{idx}. {mtype.upper()}")
match_type_choice = int(input("Enter choice: ").strip()) - 1
match_type = match_types[match_type_choice]

# Confirm choices
print(f"\nGender selected: {gender.capitalize()}")
print(f"Match Type selected: {match_type.upper()}")

# Build path and load file
file_path = os.path.join(gender_path, f"{match_type}_{gender}_match_summary_cleaned.csv")
match_summary = pd.read_csv(file_path)

# Select relevant columns
summary = match_summary[['match_id', 'season', 'match_date', 'event_name', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'ground']].copy()

# Display
summary


Select Gender:
1. Female
2. Male

Select Match Type:
1. IT20
2. MDM
3. ODI
4. ODM
5. T20
6. TEST

Gender selected: Male
Match Type selected: T20


,match_id,season,match_date,event_name,team1,team2,toss_winner,toss_decision,winner,ground
0,1486582,2025,2025-05-21,Bangladesh tour of United Arab Emirates,United Arab Emirates,Bangladesh,United Arab Emirates,field,United Arab Emirates,"Sharjah Cricket Association Stadium, Sharjah"
1,1486228,2025,2025-05-18,Slovenia tour of Austria,Slovenia,Austria,Austria,bat,Austria,"Ballpark Ground, Graz"
2,1486227,2025,2025-05-18,Slovenia tour of Austria,Slovenia,Austria,Slovenia,bat,Austria,"Ballpark Ground, Graz"
3,1486225,2025,2025-05-17,Slovenia tour of Austria,Slovenia,Austria,Slovenia,field,Austria,"Velden Cricket Ground, Latschach"
4,1484052,2025,2025-05-19,Bangladesh tour of United Arab Emirates,United Arab Emirates,Bangladesh,United Arab Emirates,field,United Arab Emirates,"Sharjah Cricket Association Stadium, Sharjah"
...,...,...,...,...,...,...,...,...,...,...
8595,225271,2006,2006-06-15,NaN,Sri Lanka,England,Sri Lanka,bat,Sri Lanka,"The Rose Bowl, Southampton"
8596,225263,2006,2006-08-28,NaN,Pakistan,England,England,bat,Pakistan,"County Ground, Bristol"
8597,222678,2005,2005-10-21,NaN,South Africa,New Zealand,New Zealand,field,New Zealand,"New Wanderers Stadium, Johannesburg"
8598,211048,2005,2005-02-17,NaN,New Zealand,Australia,Australia,bat,Australia,"Eden Park, Auckland"


### **Step 2: Unique Event Names**

In [2]:
# Get unique events and count
unique_events = summary["event_name"].unique()
num_events = summary["event_name"].nunique()

# Save unique events to CSV
events_df = pd.DataFrame({"Unique Events": unique_events})
events_df.to_csv(rf"D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\{gender}\{gender}_unique_events_summary.csv", index=False)

print("No of Events:", num_events)

No of Events: 356


### **Step 3: Swap Teams to Maintain Consistent Order**

In [3]:
# Create a column for sorted team tuples
summary["team_tuple"] = summary.apply(lambda row: tuple(sorted([row["team1"], row["team2"]])), axis=1)

# Identify tuples that appear in reverse order and swap team names where necessary
reverse_tuples = summary["team_tuple"].value_counts()[summary["team_tuple"].value_counts() > 1].index
for team_tuple in reverse_tuples:
    team1, team2 = team_tuple
    summary.loc[(summary["team1"] == team1) & (summary["team2"] == team2), ["team1", "team2"]] = team2, team1

# Drop the temporary column and filter the dataset
summary.drop(columns=["team_tuple"], inplace=True)
summary = summary[(summary["winner"] == summary["team1"]) | (summary["winner"] == summary["team2"])].sort_values(by="match_id", ascending=False).reset_index(drop=True)
summary.reset_index().sort_values(by=["match_date", "match_id"], ascending=False, inplace=True)
summary[:10]

,match_id,season,match_date,event_name,team1,team2,toss_winner,toss_decision,winner,ground
0,1486582,2025,2025-05-21,Bangladesh tour of United Arab Emirates,United Arab Emirates,Bangladesh,United Arab Emirates,field,United Arab Emirates,"Sharjah Cricket Association Stadium, Sharjah"
1,1486228,2025,2025-05-18,Slovenia tour of Austria,Slovenia,Austria,Austria,bat,Austria,"Ballpark Ground, Graz"
2,1486227,2025,2025-05-18,Slovenia tour of Austria,Slovenia,Austria,Slovenia,bat,Austria,"Ballpark Ground, Graz"
3,1486225,2025,2025-05-17,Slovenia tour of Austria,Slovenia,Austria,Slovenia,field,Austria,"Velden Cricket Ground, Latschach"
4,1484052,2025,2025-05-19,Bangladesh tour of United Arab Emirates,United Arab Emirates,Bangladesh,United Arab Emirates,field,United Arab Emirates,"Sharjah Cricket Association Stadium, Sharjah"
5,1484051,2025,2025-05-17,Bangladesh tour of United Arab Emirates,United Arab Emirates,Bangladesh,United Arab Emirates,field,Bangladesh,"Sharjah Cricket Association Stadium, Sharjah"
6,1482830,2025,2025-05-07,Estonia tour of Malta,Malta,Estonia,Estonia,field,Malta,"Marsa Sports Club, Marsa"
7,1482829,2025,2025-05-06,Estonia tour of Malta,Malta,Estonia,Estonia,field,Estonia,"Marsa Sports Club, Marsa"
8,1482828,2025,2025-05-06,Estonia tour of Malta,Malta,Estonia,Estonia,field,Malta,"Marsa Sports Club, Marsa"
9,1482827,2025,2025-05-14,Cook Islands tour of Japan,Japan,Cook Islands,Cook Islands,field,Japan,"Sano International Cricket Ground, Sano"


### **Step 4: Head-to-Head Winning Count**

In [4]:
# Calculate total matches played and head-to-head wins for team1
summary['h2h'] = summary.groupby(['team1', 'team2'])['match_id'].transform('count')
h2h_wins = summary.groupby(['team1', 'team2']).apply(lambda x: (x['winner'] == x['team1']).sum()).reset_index(name='h2h_team1_wins')

# Merge calculated wins back to the original DataFrame
summary = summary.merge(h2h_wins, on=['team1', 'team2'], how='left')
summary["h2h_team1_wins%"] = round((summary["h2h_team1_wins"] / summary["h2h"]) * 100).fillna(0).astype(int)

# Display the updated DataFrame
summary[['match_id', 'team1', 'team2', 'h2h', 'h2h_team1_wins']]

C:\Users\harik\AppData\Local\Temp\ipykernel_30052\3526048767.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  h2h_wins = summary.groupby(['team1', 'team2']).apply(lambda x: (x['winner'] == x['team1']).sum()).reset_index(name='h2h_team1_wins')


,match_id,team1,team2,h2h,h2h_team1_wins
0,1486582,United Arab Emirates,Bangladesh,6,2
1,1486228,Slovenia,Austria,4,0
2,1486227,Slovenia,Austria,4,0
3,1486225,Slovenia,Austria,4,0
4,1484052,United Arab Emirates,Bangladesh,6,2
...,...,...,...,...,...
8595,225271,Sri Lanka,England,14,4
8596,225263,Pakistan,England,30,9
8597,222678,South Africa,New Zealand,15,11
8598,211048,New Zealand,Australia,19,6


### **Step 5: Head-to-Head Winning Count at a Venue**

In [7]:
# Standardize text data
summary['team1'] = summary['team1'].str.strip().str.upper()
summary['team2'] = summary['team2'].str.strip().str.upper()
summary['ground'] = summary['ground'].str.strip().str.title().fillna('Unknown')
summary['winner'] = summary['winner'].str.strip().str.upper()

# Calculate matches played and wins at the same venue for team1
summary['h2h_venue'] = summary.groupby(['team1', 'team2', 'ground'])['match_id'].transform('count')
head_to_head = summary.groupby(['team1', 'team2', 'ground', 'winner']).size().unstack(fill_value=0)

# Create a column for wins by team1 at the same venue
def get_team1_wins(row):
    try:
        return head_to_head.loc[(row['team1'], row['team2'], row['ground']), row['team1']]
    except KeyError:
        return 0

summary['h2h_venue_team1_wins'] = summary.apply(get_team1_wins, axis=1)
summary['h2h_venue_team1_wins%'] = round((summary['h2h_venue_team1_wins'] / summary['h2h_venue']) * 100, 2).fillna(0).astype(int)

# Display the updated DataFrame
summary[['match_id', 'match_date', 'team1', 'team2', 'ground', 'winner', 'h2h_venue', 'h2h_venue_team1_wins', 'h2h_venue_team1_wins%']].head()


,match_id,match_date,team1,team2,ground,winner,h2h_venue,h2h_venue_team1_wins,h2h_venue_team1_wins%
0,1486582,2025-05-21,UNITED ARAB EMIRATES,BANGLADESH,"Sharjah Cricket Association Stadium, Sharjah",UNITED ARAB EMIRATES,3,2,66
1,1486228,2025-05-18,SLOVENIA,AUSTRIA,"Ballpark Ground, Graz",AUSTRIA,2,0,0
2,1486227,2025-05-18,SLOVENIA,AUSTRIA,"Ballpark Ground, Graz",AUSTRIA,2,0,0
3,1486225,2025-05-17,SLOVENIA,AUSTRIA,"Velden Cricket Ground, Latschach",AUSTRIA,1,0,0
4,1484052,2025-05-19,UNITED ARAB EMIRATES,BANGLADESH,"Sharjah Cricket Association Stadium, Sharjah",UNITED ARAB EMIRATES,3,2,66


In [12]:
cskVSgt = summary[((summary["team1"] == "CSK") & (summary["team2"] == "GT") | (summary["team1"] == "GT") & (summary["team2"] == "CSK")) 
                 & (summary["ground"] == "Narendra Modi Stadium, Ahmedabad")].reset_index(drop=True)
cskVSgt

,match_id,season,match_date,event_name,team1,team2,toss_winner,toss_decision,winner,ground,...,h2h_venue_team1_wins,h2h_venue_team1_wins%,team1_venue,team2_venue,team1_venue_wins,team2_venue_wins,team1_venue_wins%,team2_venue_wins%,team1_last_5_wins,team2_last_5_wins
0,1426297,2024,2024-05-10,IPL,GT,CSK,CSK,field,GT,"Narendra Modi Stadium, Ahmedabad",...,2,66,6,3,2,1,33,33,3,2
1,1370353,2023,2023-05-29,IPL,GT,CSK,CSK,field,CSK,"Narendra Modi Stadium, Ahmedabad",...,2,66,6,3,2,1,33,33,4,4
2,1359475,2023,2023-03-31,IPL,GT,CSK,GT,field,GT,"Narendra Modi Stadium, Ahmedabad",...,2,66,6,3,2,1,33,33,2,1


### **Step 6: Team Wins at a Venue**

In [13]:
# Calculate matches played at the same venue for team1 and team2
summary['team1_venue'] = summary.groupby(['team1', 'ground'])['match_id'].transform('count')
summary['team2_venue'] = summary.groupby(['team2', 'ground'])['match_id'].transform('count')

# Function to get team wins
def get_team_wins(row, team_col):
    try:
        return head_to_head.loc[(row['team1'], row['team2'], row['ground']), row[team_col]]
    except KeyError:
        return 0
    except:
        return 0

# Apply the function to calculate wins at the venue
summary['team1_venue_wins'] = summary.apply(lambda row: get_team_wins(row, 'team1'), axis=1)
summary['team2_venue_wins'] = summary.apply(lambda row: get_team_wins(row, 'team2'), axis=1)

# Calculate win percentages
summary['team1_venue_wins%'] = round((summary['team1_venue_wins'] / summary['team1_venue']) * 100, 2).fillna(0).astype(int)
summary['team2_venue_wins%'] = round((summary['team2_venue_wins'] / summary['team2_venue']) * 100, 2).fillna(0).astype(int)

# Display the updated DataFrame
display_cols = ['match_id', 'match_date', 'team1', 'team2', 'team1_venue_wins', 'team1_venue_wins%', 'team2_venue_wins', 'team2_venue_wins%', 'ground', 'winner']
summary[display_cols].head()


,match_id,match_date,team1,team2,team1_venue_wins,team1_venue_wins%,team2_venue_wins,team2_venue_wins%,ground,winner
0,1486582,2025-05-21,UNITED ARAB EMIRATES,BANGLADESH,2,66,1,20,"Sharjah Cricket Association Stadium, Sharjah",UNITED ARAB EMIRATES
1,1486228,2025-05-18,SLOVENIA,AUSTRIA,0,0,2,100,"Ballpark Ground, Graz",AUSTRIA
2,1486227,2025-05-18,SLOVENIA,AUSTRIA,0,0,2,100,"Ballpark Ground, Graz",AUSTRIA
3,1486225,2025-05-17,SLOVENIA,AUSTRIA,0,0,1,100,"Velden Cricket Ground, Latschach",AUSTRIA
4,1484052,2025-05-19,UNITED ARAB EMIRATES,BANGLADESH,2,66,1,20,"Sharjah Cricket Association Stadium, Sharjah",UNITED ARAB EMIRATES


### **Step 7: Last 5 Matches Performance**

In [14]:
import pandas as pd

# Function to calculate last 5 matches wins for each match
def calculate_last_5_wins(df):
    df['team1_last_5_wins'] = 0
    df['team2_last_5_wins'] = 0
    
    teams = set(df['team1']).union(set(df['team2']))
    for team in teams:
        team_matches = df[(df['team1'] == team) | (df['team2'] == team)].sort_values(by=['match_date', 'match_id'], ascending=False)
        wins_count = []
        for i in range(len(team_matches)):
            match = team_matches.iloc[i]
            prev_matches = team_matches.iloc[i+1:i+6]  # next 5 matches in descending order
            wins = len(prev_matches[prev_matches['winner'] == team])
            wins_count.append(wins)
        
        # Ensure wins_count is aligned with the correct matches in the main DataFrame
        df.loc[df[df['team1'] == team].index, 'team1_last_5_wins'] = wins_count[:len(df[df['team1'] == team])]
        df.loc[df[df['team2'] == team].index, 'team2_last_5_wins'] = wins_count[:len(df[df['team2'] == team])]

    return df

# Calculate last 5 matches wins for each match
summary = calculate_last_5_wins(summary)

# Display the DataFrame
summary[['match_id', 'team1', 'team2', 'match_date', 'team1_last_5_wins', 'team2_last_5_wins']][:5]


,match_id,team1,team2,match_date,team1_last_5_wins,team2_last_5_wins
0,1486582,UNITED ARAB EMIRATES,BANGLADESH,2025-05-21,3,4
1,1486228,SLOVENIA,AUSTRIA,2025-05-18,1,4
2,1486227,SLOVENIA,AUSTRIA,2025-05-18,1,4
3,1486225,SLOVENIA,AUSTRIA,2025-05-17,1,4
4,1484052,UNITED ARAB EMIRATES,BANGLADESH,2025-05-19,3,4


In [16]:
summary[(summary["team1"] == "CSK") | (summary["team2"] == "CSK")]

,match_id,season,match_date,event_name,team1,team2,toss_winner,toss_decision,winner,ground,...,h2h_venue_team1_wins,h2h_venue_team1_wins%,team1_venue,team2_venue,team1_venue_wins,team2_venue_wins,team1_venue_wins%,team2_venue_wins%,team1_last_5_wins,team2_last_5_wins
106,1473500,2025,2025-05-20,IPL,RR,CSK,RR,field,RR,"Arun Jaitley Stadium, Delhi",...,1,100,6,6,1,0,16,0,1,1
110,1473494,2025,2025-05-07,IPL,KKR,CSK,KKR,bat,CSK,"Eden Gardens, Kolkata",...,4,36,25,13,4,7,16,53,3,1
114,1473489,2025,2025-05-03,IPL,RCB,CSK,CSK,field,RCB,"M Chinnaswamy Stadium, Bengaluru",...,6,54,71,12,6,5,8,41,4,1
117,1473486,2025,2025-04-30,IPL,PBKS,CSK,PBKS,field,PBKS,"Ma Chidambaram Stadium, Chepauk, Chennai",...,5,55,10,77,5,4,50,5,3,1
122,1473480,2025,2025-04-25,IPL,SRH,CSK,SRH,field,SRH,"Ma Chidambaram Stadium, Chepauk, Chennai",...,3,30,17,77,3,7,17,9,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8529,336001,2008,2008-05-02,IPL,DC,CSK,CSK,bat,DC,"Ma Chidambaram Stadium, Chepauk, Chennai",...,3,30,10,77,3,7,30,9,3,4
8534,335996,2008,2008-04-28,IPL,RCB,CSK,CSK,bat,CSK,"M Chinnaswamy Stadium, Bengaluru",...,6,54,71,12,6,5,8,41,3,3
8537,335993,2008,2008-04-26,IPL,KKR,CSK,KKR,bat,CSK,"Ma Chidambaram Stadium, Chepauk, Chennai",...,4,33,12,77,4,8,33,10,2,2
8541,335989,2008,2008-04-23,IPL,MI,CSK,MI,field,CSK,"Ma Chidambaram Stadium, Chepauk, Chennai",...,5,55,12,77,5,4,41,5,3,1


### **Step 8: Event Name Standardization**

In [161]:
# Standardize event column based on team participation
team_event_counts = summary.groupby(['team1', 'team2'])['event_name'].nunique().reset_index()
multiple_events_teams = team_event_counts[team_event_counts['event_name'] > 1]
teams_in_multiple_events = set(multiple_events_teams['team1']).union(set(multiple_events_teams['team2']))

# List of events to be considered as T20I
t20i_events = [
    'T20I',
    'Central American Cricket Championships',
    'South American Men\'s Championships'
]

# Apply the condition to set the 'event' column
if match_type == "T20":
    summary['event'] = summary.apply(lambda row: 'T20I' if row['event_name'] in t20i_events or row['team1'] in teams_in_multiple_events or row['team2'] 
                                     in teams_in_multiple_events else row['event_name'], axis=1)

# Check if the match type is "ODI"
elif match_type == "ODI":
    summary['event'] = summary.apply(lambda row: 'ODI' if row['team1'] in teams_in_multiple_events or row['team2'] in teams_in_multiple_events else row['event_name'],axis=1)

# Check if the match type is "ODM"
elif match_type == "ODM":
    summary['event'] = summary.apply(lambda row: 'ODM' if row['team1'] in teams_in_multiple_events or row['team2'] in teams_in_multiple_events else row['event_name'],axis=1)

# Check if the match type is "MDM"
elif match_type == "MDM":
    summary['event'] = summary.apply(lambda row: 'MDM' if row['team1'] in teams_in_multiple_events or row['team2'] in teams_in_multiple_events else row['event_name'],axis=1)

# If none of the above match types, assume it is a Test match
elif match_type == "Test":
    summary['event'] = summary.apply(lambda row: 'Test' if row['team1'] in teams_in_multiple_events or row['team2'] in teams_in_multiple_events else row['event_name'],axis=1)

# Display the first 10 rows of the summary DataFrame to check the updates
summary[:5]


,match_id,season,match_date,event_name,team1,team2,toss_winner,toss_decision,winner,ground,...,h2h_venue_team1_wins%,team1_venue,team2_venue,team1_venue_wins,team2_venue_wins,team1_venue_wins%,team2_venue_wins%,team1_last_5_wins,team2_last_5_wins,event
0,1476410,2025,2025-04-30,Zimbabwe tour of Bangladesh,Zimbabwe,Bangladesh,Zimbabwe,bat,Bangladesh,"Zahur Ahmed Chowdhury Stadium, Chittagong, Cha...",...,0,2,13,0,2,0,15,1,1,Test
1,1476409,2025,2025-04-23,Zimbabwe tour of Bangladesh,Zimbabwe,Bangladesh,Bangladesh,bat,Zimbabwe,"Sylhet International Cricket Stadium, Academy ...",...,100,2,4,2,0,100,0,0,1,Test
2,1467700,2025,2025-02-10,Ireland tour of Zimbabwe,Zimbabwe,Ireland,Ireland,bat,Ireland,"Queens Sports Club, Bulawayo",...,0,10,1,0,1,0,100,0,1,Test
3,1459907,2025,2025-02-09,Australia tour of Sri Lanka,Sri Lanka,Australia,Sri Lanka,bat,Australia,"Galle International Stadium, Galle",...,28,32,7,2,5,6,71,2,4,Test
4,1459906,2025,2025-02-01,Australia tour of Sri Lanka,Sri Lanka,Australia,Australia,bat,Australia,"Galle International Stadium, Galle",...,28,32,7,2,5,6,71,3,4,Test


In [162]:
summary['event'].unique()

array(['Test'], dtype=object)

In [163]:
summary[summary['event'] == "Big Bash League"]

,match_id,season,match_date,event_name,team1,team2,toss_winner,toss_decision,winner,ground,...,h2h_venue_team1_wins%,team1_venue,team2_venue,team1_venue_wins,team2_venue_wins,team1_venue_wins%,team2_venue_wins%,team1_last_5_wins,team2_last_5_wins,event


### **Step 9: Capitalizing IPL teams**

In [164]:
# Create a mask for rows where event_name is "IPL"
mask = summary["event_name"] == "IPL"

# Convert the specified columns to uppercase only for the rows where event_name is "IPL"
summary.loc[mask, 'team1'] = summary.loc[mask, 'team1'].str.upper()
summary.loc[mask, 'team2'] = summary.loc[mask, 'team2'].str.upper()
summary.loc[mask, 'toss_winner'] = summary.loc[mask, 'toss_winner'].str.upper()
summary.loc[mask, 'winner'] = summary.loc[mask, 'winner'].str.upper()

### **Step 10: Save Cleaned DataFrame**

In [165]:
# Select and reorder final columns
summary_df = summary[['match_id', 'season', 'event_name', 'event', 'match_date', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'h2h_team1_wins%', 'h2h_venue_team1_wins%', 'team1_venue_wins%', 'team2_venue_wins%', 'team1_last_5_wins', 'team2_last_5_wins', 'ground']].copy()

# Save the cleaned summary DataFrame to CSV
summary_df.to_csv(fr"D:\GITHUB\Predictive-Analytics-for-Cricket-Matches-Using-Machine-Learning\source_data\Summary\match_summary\{gender}\{match_type}_{gender}_match_summary_cleaned.csv", index=False)

summary_df.head()

,match_id,season,event_name,event,match_date,team1,team2,toss_winner,toss_decision,winner,h2h_team1_wins%,h2h_venue_team1_wins%,team1_venue_wins%,team2_venue_wins%,team1_last_5_wins,team2_last_5_wins,ground
0,1476410,2025,Zimbabwe tour of Bangladesh,Test,2025-04-30,Zimbabwe,Bangladesh,Zimbabwe,bat,Bangladesh,33,0,0,15,1,1,"Zahur Ahmed Chowdhury Stadium, Chittagong, Cha..."
1,1476409,2025,Zimbabwe tour of Bangladesh,Test,2025-04-23,Zimbabwe,Bangladesh,Bangladesh,bat,Zimbabwe,33,100,100,0,0,1,"Sylhet International Cricket Stadium, Academy ..."
2,1467700,2025,Ireland tour of Zimbabwe,Test,2025-02-10,Zimbabwe,Ireland,Ireland,bat,Ireland,0,0,0,100,0,1,"Queens Sports Club, Bulawayo"
3,1459907,2025,Australia tour of Sri Lanka,Test,2025-02-09,Sri Lanka,Australia,Sri Lanka,bat,Australia,25,28,6,71,2,4,"Galle International Stadium, Galle"
4,1459906,2025,Australia tour of Sri Lanka,Test,2025-02-01,Sri Lanka,Australia,Australia,bat,Australia,25,28,6,71,3,4,"Galle International Stadium, Galle"
